In [5]:
import msgpack
import database
from sqlalchemy import select

In [6]:
map_file = "output/07_create_article_project_map/2017-03-27 14:10:49 77c76e1/articles_projects.m"
out_file = "biography_articles.tsv"

In [7]:
with open(map_file, 'rb') as f:
    article_projects = msgpack.unpackb(f.read())

In [8]:
conn = database.engine.connect()
tab = database.schema.article_name_id
article_count = 0
name_count = 0
talk_count = 0
with open(out_file, "wb") as out:
    out.write("article_id\tarticle_name\tfrom_timestamp\tto_timestamp\n")
    for article_id, projects in article_projects.iteritems():
        if 256 in projects:
            article_count += 1
            stmt = select([tab.c.article_name, tab.c.from_ts, tab.c.to_ts]) \
                .where(tab.c.article_id == article_id)
            result = conn.execute(stmt)
            name_count += result.rowcount
            try:
                for d in result:
                    article_name = d[0]
                    if article_name[0:5] == "Talk:":
                        talk_count += 1
                        continue
                    from_ts = d[1]
                    to_ts = d[2]
                    out.write("%d\t%s\t%d\t%d\n" % (
                        article_id,
                        article_name.encode('utf-8'),
                        from_ts,
                        to_ts
                    ))
            except TypeError:
                print "TypeError: %d" % article_id
                continue
print article_count
print name_count
print talk_count
print "done"

1727372
2218654
1040995
done


In [9]:
name_count - talk_count

1177659L